# mix produttivo

Un’azienda utilizza tre reparti (1, 2 e 3) per produrre due composti (A e B).

Ogni kg di A e B richiede un dato numero di ore di lavorazione in ciascun reparto e ogni reparto ha una data capacità produttiva, ossia è disponibile per un certo numero di ore al giorno (vedi schema).

![img](./img.png)

![tab](./tab.png)

Considerato che ogni composto ha un dato prezzo di vendita al kg (ultima colonna della tabella), qual è la produzione di A e B che, nel rispetto delle capacità produttive dei reparti, massimizza il profitto dell’azienda?


## formulazione generale del problema di mix produttivo

Un’azienda produce n articoli $P_1$,...,$P_n$ utilizzando un impianto con $m$ reparti $R_1$,...,$R_m$.

Ogni unità dell’articolo $P_i$ assicura all’azienda un profitto netto di $p_i$ € e richiede $a_{ij}$ ore di lavorazione nel reparto $R_j$.

Qual è il mix produttivo giornaliero più redditizio per l’azienda, cioè quali sono le quantità di ogni articolo che l’azienda deve produrre per massimizzare il profitto rispettando le capacità produttive dei reparti?


## modello parametrico

![param1](./param1.png)

![param2](./param2.png)

In [3]:
from pyomo.environ import *
from pyomo.opt import SolverStatus, TerminationCondition

model = ConcreteModel()
model.name = "mix produttivo"

i = [ 'composto A', 'composto B' ]
p = [            5,             4]

j = [ 'rep 1', 'rep 2', 'rep 3']
b = [       9,      10,       8]

a = [[2, 3], 
     [1, 2], 
     [3, 1]]

model.i = Set(initialize=i)
model.j = Set(initialize=j)

a_dict = {}
for j, mj in enumerate(model.j):
    for i, mi in enumerate(model.i):
        a_dict[mj, mi] = a[j][i]

p = {mi: p[i] for i, mi in enumerate(model.i)}
b = {mj: b[j] for j, mj in enumerate(model.j)}

model.p = Param(model.i, initialize=p)
model.b = Param(model.j, initialize=b)

model.a = Param(model.j, model.i, initialize=a_dict)

model.x = Var(model.i, domain=NonNegativeReals, initialize=0)

obj_expr = sum(model.p[i]*model.x[i] for i in model.i)

model.cost = Objective(expr = obj_expr, sense=maximize)

model.constraints = ConstraintList()

for j in model.j:
    model.constraints.add(expr = sum(model.a[j, i]*model.x[i] for i in model.i) <= model.b[j])

results = SolverFactory('glpk').solve(model)

model.display()


Model mix produttivo

  Variables:
    x : Size=2, Index=i
        Key        : Lower : Value            : Upper : Fixed : Stale : Domain
        composto A :     0 : 2.14285714285714 :  None : False : False : NonNegativeReals
        composto B :     0 : 1.57142857142857 :  None : False : False : NonNegativeReals

  Objectives:
    cost : Size=1, Index=None, Active=True
        Key  : Active : Value
        None :   True : 16.999999999999982

  Constraints:
    constraints : Size=3
        Key : Lower : Body             : Upper
          1 :  None : 8.99999999999999 :   9.0
          2 :  None : 5.28571428571428 :  10.0
          3 :  None : 7.99999999999999 :   8.0
